Import das bibliotecas necessárias


In [7]:
import psycopg2 as pg
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

Configuração da conexão com o banco de dados PostgreSQL

In [8]:
try:
  conn = pg.connect(database="projeto-fbd",host="localhost",port=5432,user='postgres',password='198019')
  print('conexão concluida')
except(Exception) as error:
  #print(error)
  print('conexão não concluida')

conexão concluida


Criação de widgets

In [9]:
try:
    meta_calorica = widgets.FloatText(disabled=False, description='Meta calórica', value=None)
    quantidade_refeicoes = widgets.IntText(disabled=False, description='Quantidade', value=None)
    intervalo_refeicoes = widgets.IntText(disabled=False, description='Intervalo', value=None)
    objetivo = widgets.Text(disabled=False, placeholder='Objetivo da Dieta', value='', description='Objetivo')
    # Dúvidas em relação a pK e fK
    dieta_id = widgets.IntText(disabled=False, description='ID', value=None)
    cpf_usuario = widgets.Text(disabled=False, placeholder='CPF: ', value='', description='CPF')
    alimentos = widgets.Textarea(disabled=False, value='', placeholder='Alimentos: ', description='Alimentos')
except Exception as error:
    print('Falha:', error)


Criação dos botões e declaração das suas funções


In [10]:
#Criação dos botões
createButton = widgets.Button(description="Cadastrar")
readButton = widgets.Button(description="Consultar", value='Preencha os campos acima para consultar')
updateButton = widgets.Button(description="Atualizar", value= 'Preencha os campos acima para atualizar')
deleteButton = widgets.Button(description="Deletar", value= 'Preencha os campos acima para deletar')
output = widgets.Output()

def onCreateButton():
    try:
        cursor = conn.cursor()
        cursor.execute('insert into Dieta(Meta_Calorica,Quantidade_de_refeicoes,Intervalo_entre_refeicoes,Objetivo,DietaId,CpfUsuario) values (%f,%d,%d,%s,%d,%s)',(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario))
        cursor.execute('insert into Dieta_Alimentos(Alimentos,DietaID) values (%s,%d)',(alimentos,dieta_id))
        conn.commit()
        print('Dados inseridos com sucesso!')

        df = pd.read_sql_query('select d.*, a.alimentos from Dieta d join Dieta_Alimentos a on a.DietaId = d.DietaId',conn)

        display(df)
    except(Exception,pg.Error) as error:
        cursor.execute('ROLLBACK')
        output.clear_output()
        display(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario,alimentos)
        print('Falha ao inserir dados',error)
    finally:
        cursor.close()

def onReadButton():
    try:
        cursor = conn.cursor()
        cursor.execute('select d.*, a.alimentos from Dieta d join Dieta_Alimentos a on a.DietaId = d.DietaId')
        conn.commit
        df = pd.read_sql_query('select d.*, a.alimentos from Dieta d join Dieta_Alimentos a on a.DietaId = d.DietaId',conn)
        output.clear_output(wait=True)
        display(df)
    except(Exception,pg.Error) as error:
        conn.rollback()
        output.clear_output()
        print('falha ao ler dados',error)
    finally: 
        cursor.close()

def onUpdateButton():
    try:
        cursor = conn.cursor()
        cursor.execute('update Dieta set Meta_Calorica = %f, Quantidade_de_refeicoes = %d, Intervalo_entre_refeicoes = %d, Objetivo = %s, DietaId = %d, CpfUsuario = %s',(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario))
        cursor.execute('update Dieta_Alimentos set Alimentos = %s, DietaID = %d',(alimentos,dieta_id))
        df = pd.read_sql_query('select d.*, a.alimentos from Dieta d join Dieta_Alimentos a on a.DietaId = d.DietaId',conn)
        output.clear_output()
        display(df)
    except(Exception, pg.Error) as error:
        conn.rollback()
        output.clear_output()
        display(df)
        print('Falha ao atualizar dados',error)
    finally:
        cursor.close()

def onDeleteButton():
    try:
        cursor = conn.cursor()
        cursor.execute('delete from Dieta where DietaId = %d',(dieta_id))
        cursor.execute('delete from Dieta_Alimentos where DietaId = %d',(dieta_id))
        df = pd.read_sql_query('select d.*, a.alimentos from Dieta d join Dieta_Alimentos a on a.DietaId = d.DietaId',conn)
        output.clear_output()
        display(df)
    except(Exception, pg.Error) as error:
        conn.rollback()
        output.clear_output()
        display(df)
        print('Falha ao deletar dados',error)
    finally:
        cursor.close()

createButton.on_click(onCreateButton)
readButton.on_click(onReadButton)
updateButton.on_click(onUpdateButton)
deleteButton.on_click(onDeleteButton)

Exibir widgets

In [11]:
with output:
    display(meta_calorica,quantidade_refeicoes,intervalo_refeicoes,objetivo,dieta_id,cpf_usuario,alimentos,createButton,readButton,updateButton,deleteButton)
output

Output()